In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'cs231n/assignments/assignment1/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

# 全连接神经网络
在这个练习中，我们将使用模块化方法实现全连接网络。对于每一层，我们将实现一个 `forward` 和一个 `backward` 函数。`forward` 函数将接收输入、权重和其他参数，并返回一个输出和一个 `cache` 对象，如下所示：

```python
def layer_forward(x, w):
  """ 接收输入 x 和权重 w """
  # 进行一些计算 ...
  z = # ... 一些中间值
  # 进行更多计算 ...
  out = # 输出
   
  cache = (x, w, z, out) # 我们需要计算梯度的值
   
  return out, cache
```

backward pass 将接收上游导数和 `cache` 对象，并返回关于输入和权重的梯度，如下所示：

```python
def layer_backward(dout, cache):
  """
  接收 dout（损失对输出的导数）和 cache，
  并计算对输入的导数。
  """
  # 解包缓存值
  x, w, z, out = cache
  
  # 使用缓存中的值计算导数
  dx = # 损失对 x 的导数
  dw = # 损失对 w 的导数
  
  return dx, dw
```

在以这种方式实现大量层之后，我们将能够轻松地将它们组合起来构建具有不同架构的分类器。

In [ ]:
# 像往常一样，一些设置代码
from __future__ import print_function
import time
import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifiers.fc_net import *
from cs231n.data_utils import get_CIFAR10_data
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from cs231n.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # 设置绘图的默认大小
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# 用于自动重新加载外部模块
# 参见 http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ 返回相对误差 """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [ ]:
# 加载（预处理的）CIFAR10 数据。

data = get_CIFAR10_data()
for k, v in list(data.items()):
  print(('%s: ' % k, v.shape))

# 仿射层：前向传播
打开文件 `cs231n/layers.py` 并实现 `affine_forward` 函数。

完成后，你可以通过运行以下命令测试你的实现：

In [ ]:
# 测试 affine_forward 函数

num_inputs = 2
input_shape = (4, 5, 6)
output_dim = 3

input_size = num_inputs * np.prod(input_shape)
weight_size = output_dim * np.prod(input_shape)

x = np.linspace(-0.1, 0.5, num=input_size).reshape(num_inputs, *input_shape)
w = np.linspace(-0.2, 0.3, num=weight_size).reshape(np.prod(input_shape), output_dim)
b = np.linspace(-0.3, 0.1, num=output_dim)

out, _ = affine_forward(x, w, b)
correct_out = np.array([[ 1.49834967,  1.70660132,  1.91485297],
                        [ 3.25553199,  3.5141327,   3.77273342]])

# 将你的输出与我们的进行比较。误差应该在 e-9 左右或更小。
print('测试 affine_forward 函数：')
print('差异: ', rel_error(out, correct_out))

# 仿射层：反向传播
现在实现 `affine_backward` 函数，并使用数值梯度检查测试你的实现。

In [ ]:
# 测试 affine_backward 函数
np.random.seed(231)
x = np.random.randn(10, 2, 3)
w = np.random.randn(6, 5)
b = np.random.randn(5)
dout = np.random.randn(10, 5)

dx_num = eval_numerical_gradient_array(lambda x: affine_forward(x, w, b)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: affine_forward(x, w, b)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: affine_forward(x, w, b)[0], b, dout)

_, cache = affine_forward(x, w, b)
dx, dw, db = affine_backward(dout, cache)

# 误差应该在 e-10 左右或更小
print('测试 affine_backward 函数：')
print('dx 误差: ', rel_error(dx_num, dx))
print('dw 误差: ', rel_error(dw_num, dw))
print('db 误差: ', rel_error(db_num, db))

# ReLU 激活：前向传播
在 `relu_forward` 函数中实现 ReLU 激活函数的前向传播，并使用以下代码测试你的实现：

In [ ]:
# 测试 relu_forward 函数

x = np.linspace(-0.5, 0.5, num=12).reshape(3, 4)

out, _ = relu_forward(x)
correct_out = np.array([[ 0.,          0.,          0.,          0.,        ],
                        [ 0.,          0.,          0.04545455,  0.13636364,],
                        [ 0.22727273,  0.31818182,  0.40909091,  0.5,       ]])

# 将你的输出与我们的进行比较。误差应该在 e-8 左右
print('测试 relu_forward 函数：')
print('差异: ', rel_error(out, correct_out))

# ReLU 激活：反向传播
现在在 `relu_backward` 函数中实现 ReLU 激活函数的反向传播，并使用数值梯度检查测试你的实现：

In [ ]:
np.random.seed(231)
x = np.random.randn(10, 10)
dout = np.random.randn(*x.shape)

dx_num = eval_numerical_gradient_array(lambda x: relu_forward(x)[0], x, dout)

_, cache = relu_forward(x)
dx = relu_backward(dout, cache)

# 误差应该在 e-12 左右
print('测试 relu_backward 函数：')
print('dx 误差: ', rel_error(dx_num, dx))

## 内联问题 1：

我们只要求你实现 ReLU，但神经网络中可以使用许多不同的激活函数，每个都有其优缺点。特别是，激活函数的一个常见问题是在反向传播期间获得零（或接近零）的梯度流。以下哪些激活函数有这个问题？如果你在一维情况下考虑这些函数，什么类型的输入会导致这种行为？
1. Sigmoid
2. ReLU
3. Leaky ReLU

$\color{blue}{\textit 你的答案：}$ *在此填写*

# "三明治"层
有一些在神经网络中经常使用的层模式。例如，仿射层之后经常跟着 ReLU 非线性。为了使这些常见模式容易使用，我们在文件 `cs231n/layer_utils.py` 中定义了几个便利层。

现在请查看 `affine_relu_forward` 和 `affine_relu_backward` 函数，并运行以下代码进行数值梯度检查：

In [ ]:
from cs231n.layer_utils import affine_relu_forward, affine_relu_backward
np.random.seed(231)
x = np.random.randn(2, 3, 4)
w = np.random.randn(12, 10)
b = np.random.randn(10)
dout = np.random.randn(2, 10)

out, cache = affine_relu_forward(x, w, b)
dx, dw, db = affine_relu_backward(dout, cache)

dx_num = eval_numerical_gradient_array(lambda x: affine_relu_forward(x, w, b)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: affine_relu_forward(x, w, b)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: affine_relu_forward(x, w, b)[0], b, dout)

# 相对误差应该在 e-10 左右或更小
print('测试 affine_relu_forward 和 affine_relu_backward：')
print('dx 误差: ', rel_error(dx_num, dx))
print('dw 误差: ', rel_error(dw_num, dw))
print('db 误差: ', rel_error(db_num, db))

# 损失层：Softmax
现在在 `cs231n/layers.py` 的 `softmax_loss` 函数中实现 Softmax 的损失和梯度。这些应该类似于你在 `cs231n/classifiers/softmax.py` 中实现的。其他损失函数（例如 `svm_loss`）也可以以模块化方式实现，但是本作业不需要。

你可以通过运行以下代码确保实现正确：

In [ ]:
np.random.seed(231)
num_classes, num_inputs = 10, 50
x = 0.001 * np.random.randn(num_inputs, num_classes)
y = np.random.randint(num_classes, size=num_inputs)


dx_num = eval_numerical_gradient(lambda x: softmax_loss(x, y)[0], x, verbose=False)
loss, dx = softmax_loss(x, y)

# 测试 softmax_loss 函数。损失应该接近 2.3，dx 误差应该在 e-8 左右
print('\n测试 softmax_loss：')
print('损失: ', loss)
print('dx 误差: ', rel_error(dx_num, dx))

# 两层网络
打开文件 `cs231n/classifiers/fc_net.py` 并完成 `TwoLayerNet` 类的实现。通读它以确保你理解 API。你可以运行下面的单元格来测试你的实现。

In [ ]:
np.random.seed(231)
N, D, H, C = 3, 5, 50, 7
X = np.random.randn(N, D)
y = np.random.randint(C, size=N)

std = 1e-3
model = TwoLayerNet(input_dim=D, hidden_dim=H, num_classes=C, weight_scale=std)

print('测试初始化 ... ')
W1_std = abs(model.params['W1'].std() - std)
b1 = model.params['b1']
W2_std = abs(model.params['W2'].std() - std)
b2 = model.params['b2']
assert W1_std < std / 10, '第一层权重看起来不对'
assert np.all(b1 == 0), '第一层偏置看起来不对'
assert W2_std < std / 10, '第二层权重看起来不对'
assert np.all(b2 == 0), '第二层偏置看起来不对'

print('测试推理时前向传播 ... ')
model.params['W1'] = np.linspace(-0.7, 0.3, num=D*H).reshape(D, H)
model.params['b1'] = np.linspace(-0.1, 0.9, num=H)
model.params['W2'] = np.linspace(-0.3, 0.4, num=H*C).reshape(H, C)
model.params['b2'] = np.linspace(-0.9, 0.1, num=C)
X = np.linspace(-5.5, 4.5, num=N*D).reshape(D, N).T
scores = model.loss(X)
correct_scores = np.asarray(
  [[11.53165108,  12.2917344,   13.05181771,  13.81190102,  14.57198434, 15.33206765,  16.09215096],
   [12.05769098,  12.74614105,  13.43459113,  14.1230412,   14.81149128, 15.49994135,  16.18839143],
   [12.58373087,  13.20054771,  13.81736455,  14.43418138,  15.05099822, 15.66781506,  16.2846319 ]])
scores_diff = np.abs(scores - correct_scores).sum()
assert scores_diff < 1e-6, '推理时前向传播出现问题'

print('测试训练时损失（无正则化）')
y = np.asarray([0, 5, 1])
loss, grads = model.loss(X, y)
correct_loss = 3.4702243556
assert abs(loss - correct_loss) < 1e-10, '训练时损失出现问题'

model.reg = 1.0
loss, grads = model.loss(X, y)
correct_loss = 26.5948426952
assert abs(loss - correct_loss) < 1e-10, '正则化损失出现问题'

# 误差应该在 e-7 左右或更小
for reg in [0.0, 0.7]:
  print('运行数值梯度检查与 reg = ', reg)
  model.reg = reg
  loss, grads = model.loss(X, y)

  for name in sorted(grads):
    f = lambda _: model.loss(X, y)[0]
    grad_num = eval_numerical_gradient(f, model.params[name], verbose=False)
    print('%s 相对误差: %.2e' % (name, rel_error(grad_num, grads[name])))

# 求解器
打开文件 `cs231n/solver.py` 并通读它以熟悉 API。完成后，使用 `Solver` 实例训练一个 `TwoLayerNet`，在验证集上达到大约 `36%` 的准确率。

In [ ]:
input_size = 32 * 32 * 3
hidden_size = 50
num_classes = 10
model = TwoLayerNet(input_size, hidden_size, num_classes)
solver = None

##############################################################################
# TODO: 使用 Solver 实例训练一个 TwoLayerNet，在验证集上达到大约 36% #
# 的准确率。                                                                #
##############################################################################

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

# 调试训练
使用我们上面提供的默认参数，你应该得到验证集上约 0.36 的验证准确率。这不是很好。

获得对问题所在深入了解的一个策略是绘制优化期间的损失函数以及训练和验证集上的准确率。

另一个策略是可视化网络第一层学到的权重。在大多数在视觉数据上训练的神经网络中，可视化时第一层权重通常显示出一些可见的结构。

In [ ]:
# 运行此单元格以可视化训练损失和训练/验证准确率

plt.subplot(2, 1, 1)
plt.title('训练损失')
plt.plot(solver.loss_history, 'o')
plt.xlabel('迭代')

plt.subplot(2, 1, 2)
plt.title('准确率')
plt.plot(solver.train_acc_history, '-o', label='训练')
plt.plot(solver.val_acc_history, '-o', label='验证')
plt.plot([0.5] * len(solver.val_acc_history), 'k--')
plt.xlabel('轮次')
plt.legend(loc='lower right')
plt.gcf().set_size_inches(15, 12)
plt.show()

In [ ]:
from cs231n.vis_utils import visualize_grid

# 可视化网络权重

def show_net_weights(net):
    W1 = net.params['W1']
    W1 = W1.reshape(3, 32, 32, -1).transpose(3, 1, 2, 0)
    plt.imshow(visualize_grid(W1, padding=3).astype('uint8'))
    plt.gca().axis('off')
    plt.show()

show_net_weights(model)

# 调整你的超参数

**出了什么问题？**。查看上面的可视化，我们看到损失大体上线性下降，这似乎表明学习率可能太低。此外，训练和验证准确率之间没有差距，这表明我们使用的模型容量较低，我们应该增加其大小。另一方面，使用非常大的模型，我们期望看到更多过拟合，这表现为训练和验证准确率之间存在很大差距。

**调整**。调整超参数并培养它们如何影响最终性能的直觉是使用神经网络的重要组成部分，所以我们希望你得到大量练习。在下面，你应该尝试不同的各种超参数值，包括隐藏层大小、学习率、训练轮数和正则化强度。你也可以考虑调整学习率衰减，但使用默认值你应该能够获得良好的性能。

**近似结果**。你应该在验证集上实现超过 48% 的分类准确率。我们的最佳网络在验证集上得到超过 52%。

**实验**：你在这个练习中的目标是在 CIFAR-10 上获得尽可能好的结果（52% 可以作为参考），使用全连接神经网络。随意实现你自己的技术（例如 PCA 降低维度，或添加 dropout，或向求解器添加功能等）。

In [ ]:
best_model = None



#################################################################################
# TODO: 使用验证集调整超参数。将你训练的最佳模型存储在
# best_model 中。                                                                #
#                                                                               #
# 为了帮助你调试网络，使用类似于我们上面使用的可视化可能会有帮助；这些可视化与我们上面为调整不佳的网络看到的会有显著的定性差异。    #
#                                                                               #
# 手动调整超参数很有趣，但你可能会发现编写代码来自动遍历超参数的组合            #
# 组合会很有用，就像我们之前做的那样。                                         #
#################################################################################

################################################################################
#                              END OF YOUR CODE                                #
################################################################################

# 测试你的模型！
运行你的最佳模型在验证和测试集上。你应该在验证集和测试集上实现超过 48% 的准确率。

In [ ]:
y_val_pred = np.argmax(best_model.loss(data['X_val']), axis=1)
print('Validation set accuracy: ', (y_val_pred == data['y_val']).mean())

In [ ]:
y_test_pred = np.argmax(best_model.loss(data['X_test']), axis=1)
print('测试集准确率: ', (y_test_pred == data['y_test']).mean())

In [ ]:
# 保存最佳模型
best_model.save("best_two_layer_net.npy")

## 内联问题 2：

现在你已经训练了一个神经网络分类器，你可能会发现测试准确率远低于训练准确率。我们可以通过哪些方式减少这种差距？选择所有适用的选项。

1. 在更大的数据集上训练。
2. 添加更多隐藏单元。
3. 增加正则化强度。
4. 以上都不对。

$\color{blue}{\textit 你的答案：}$

$\color{blue}{\textit 你的解释：}$